## Building A Chatbot
 We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.


In [1]:
import os
from dotenv import load_dotenv
load_dotenv() ## loading all the environment variables

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key



'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'

In [2]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x10f324770>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x10f325160>, model_name='Gemma2-9b-It', groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Sadiq and I am a AI Engineer")])

AIMessage(content="Hi Sadiq, it's nice to meet you!\n\nThat's an interesting field. What kind of AI engineering do you specialize in?  Do you have a favorite project you've worked on?  \n\nI'm always eager to learn more about what people are doing in the world of AI. 😄  \n\n", response_metadata={'token_usage': {'completion_tokens': 70, 'prompt_tokens': 27, 'total_tokens': 97, 'completion_time': 0.145545939, 'prompt_time': 0.002594259, 'queue_time': None, 'total_time': 0.148140198}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-5a0aff88-32d5-4e2a-b928-7b8628ca0a35-0', usage_metadata={'input_tokens': 27, 'output_tokens': 70, 'total_tokens': 97})

In [4]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Sadiq and I am a AI Engineer"),
        AIMessage(content="Hello Sadiq! It's nice to meet you. \n\nAs a Chief AI Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You are Sadiq, and you are an AI Engineer!  \n\nDid you forget for a moment? 😄 \n\nIs there anything specific you'd like to talk about regarding your work as an AI Engineer?  I'm happy to chat about AI concepts, recent advancements, or even just your day-to-day experiences.\n", response_metadata={'token_usage': {'completion_tokens': 71, 'prompt_tokens': 103, 'total_tokens': 174, 'completion_time': 0.147968729, 'prompt_time': 0.004990428, 'queue_time': None, 'total_time': 0.15295915699999998}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-24eacf36-fbde-4b6c-9069-0fcdfd717d22-0', usage_metadata={'input_tokens': 103, 'output_tokens': 71, 'total_tokens': 174})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [5]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [6]:
config={"configurable":{"session_id":"chat1"}}

In [7]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Sadiq and I am a AI Engineer")],
    config=config
)

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


In [8]:
response.content

"Hi Sadiq, it's nice to meet you!  \n\nThat's fascinating! As an AI myself, I'm always interested to connect with other people working in the field. What kind of AI engineering do you specialize in? \n\nAre you working on any cool projects you'd like to talk about?\n"

In [9]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


AIMessage(content="Your name is Sadiq, as you told me at the beginning of our conversation. 😊  \n\nIs there anything else you'd like to chat about?  I'm happy to answer any questions you have or just have a friendly conversation.\n", response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 110, 'total_tokens': 163, 'completion_time': 0.109445212, 'prompt_time': 0.006123308, 'queue_time': None, 'total_time': 0.11556852}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-320fa99d-5ec1-4605-8c37-33bdd1339699-0', usage_metadata={'input_tokens': 110, 'output_tokens': 53, 'total_tokens': 163})

In [10]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


"As an AI, I have no memory of past conversations and I don't know your name. If you'd like to tell me, I'm happy to remember it! 😊  \n\n"

In [11]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


"Hi John! It's nice to meet you. 👋 \n\nWhat can I do for you today? \n"

In [12]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


'Your name is John!  Remember I met you a moment ago.  😄  \n\nIs there anything else I can help you with, John?\n'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [13]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [14]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Sadiq")]})

AIMessage(content="Hello Sadiq, it's nice to meet you!  \n\nHow can I help you today? 😊 \n\n", response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 37, 'total_tokens': 64, 'completion_time': 0.054941272, 'prompt_time': 0.002889939, 'queue_time': None, 'total_time': 0.057831211}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-d9250261-fbe7-4707-8105-82d07a4252fc-0', usage_metadata={'input_tokens': 37, 'output_tokens': 27, 'total_tokens': 64})

In [15]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [16]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Sadiq")],
    config=config
)

response

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


AIMessage(content="Hello Sadiq, it's nice to meet you!\n\nHow can I help you today? 😊  \n", response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 37, 'total_tokens': 62, 'completion_time': 0.050568002, 'prompt_time': 0.004250239, 'queue_time': None, 'total_time': 0.054818241000000004}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-82f70050-7952-4e6f-98a8-77f1bb74866a-0', usage_metadata={'input_tokens': 37, 'output_tokens': 25, 'total_tokens': 62})

In [17]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


'Your name is Sadiq. 😊  I remember!  \n\n'

In [18]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [19]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Sadiq")],"language":"Hindi"})
response.content

'Namaste Sadiq! 😊  आपका नाम सुनाकर अच्छा लगा।  \n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [20]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [21]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Sadiq")],"language":"Hindi"},
    config=config
)
repsonse.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


'नमस्ते Sadiq! 👋 \n\nआपसे मिलकर अच्छा लगा। क्या मैं आपकी कोई मदद कर सकता हूँ? 😊\n'

In [22]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


In [23]:
response.content

'आपका नाम Sadiq है।  😊 \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [24]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

/opt/anaconda3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


[SystemMessage(content="you're a good assistant"),
 HumanMessage(content='I like vanilla ice cream'),
 AIMessage(content='nice'),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

In [25]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As a large language model, I don't have access to your personal information, including your ice cream preferences! \n\nWhat's your favorite flavor?  🍦😊  \n"

In [26]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked what 2 + 2 equals.  \n\n\n\nLet me know if you want to try another one!\n'

In [27]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [28]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


"As an AI, I don't have access to past conversations or any personal information about you, including your name.  \n\nIf you'd like to tell me your name, I'd be happy to use it! 😊\n"

In [29]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


"As a large language model, I have no memory of past conversations. If you'd like to ask me a math problem, I'm happy to help! 😊  Just let me know what it is. \n\n"